In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from config import gkey 

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
popcities = pd.read_csv('sorted_population_final.csv')
popcities

,Unnamed: 0,City,State,2018 Population,2019 Population,2020 Population,2018-2019 Population Rate,2019-2020 Population Rate,Diff in Pop Rate 2018-2020
0,168,Salt Lake City,UT,200435,200567,213367,0.065857,6.381907,6.316051
1,64,Hialeah,FL,234941,233339,243208,-0.681873,4.229469,4.911342
2,111,Jersey City,NJ,263851,262075,271099,-0.673107,3.443289,4.116396
3,27,Irvine,CA,281732,287401,303956,2.012196,5.760244,3.748048
4,169,Alexandria,VA,159069,159428,165748,0.225688,3.964172,3.738484
...,...,...,...,...,...,...,...,...,...
177,10,Surprise,AZ,137717,141664,142049,2.866022,0.271770,-2.594253
178,11,Tempe,AZ,191746,195805,194218,2.116863,-0.810500,-2.927363
179,121,Fayetteville,NC,210748,211657,205646,0.431321,-2.839972,-3.271293
180,163,Midland,TX,142173,146038,145012,2.718519,-0.702557,-3.421076


In [4]:
# Configure gmaps
gmaps.configure(api_key=gkey)

,Unnamed: 0,City,State,2018 Population,2019 Population,2020 Population,2018-2019 Population Rate,2019-2020 Population Rate,Diff in Pop Rate 2018-2020,"City, State",CityState
0,168,Salt Lake City,UT,200435,200567,213367,0.065857,6.381907,6.316051,"Salt Lake City , UT","Salt Lake City , UT"
1,64,Hialeah,FL,234941,233339,243208,-0.681873,4.229469,4.911342,"Hialeah , FL","Hialeah , FL"
2,111,Jersey City,NJ,263851,262075,271099,-0.673107,3.443289,4.116396,"Jersey City , NJ","Jersey City , NJ"
3,27,Irvine,CA,281732,287401,303956,2.012196,5.760244,3.748048,"Irvine , CA","Irvine , CA"
4,169,Alexandria,VA,159069,159428,165748,0.225688,3.964172,3.738484,"Alexandria , VA","Alexandria , VA"
...,...,...,...,...,...,...,...,...,...,...,...
177,10,Surprise,AZ,137717,141664,142049,2.866022,0.271770,-2.594253,"Surprise , AZ","Surprise , AZ"
178,11,Tempe,AZ,191746,195805,194218,2.116863,-0.810500,-2.927363,"Tempe , AZ","Tempe , AZ"
179,121,Fayetteville,NC,210748,211657,205646,0.431321,-2.839972,-3.271293,"Fayetteville , NC","Fayetteville , NC"
180,163,Midland,TX,142173,146038,145012,2.718519,-0.702557,-3.421076,"Midland , TX","Midland , TX"


In [29]:
pull = ["City", "State", "Diff in Pop Rate 2018-2020"]
pull_pop = popcities[pull]
pull_pop


,City,State,Diff in Pop Rate 2018-2020
0,Salt Lake City,UT,6.316051
1,Hialeah,FL,4.911342
2,Jersey City,NJ,4.116396
3,Irvine,CA,3.748048
4,Alexandria,VA,3.738484
...,...,...,...
177,Surprise,AZ,-2.594253
178,Tempe,AZ,-2.927363
179,Fayetteville,NC,-3.271293
180,Midland,TX,-3.421076


In [30]:
# Add columns for lat, lng,
# Hint: Be sure to specify "" as the initial value for each column.

pull_pop['Lat'] = ""
pull_pop['Lng'] = ""
pull_pop

,City,State,Diff in Pop Rate 2018-2020,Lat,Lng
0,Salt Lake City,UT,6.316051,,
1,Hialeah,FL,4.911342,,
2,Jersey City,NJ,4.116396,,
3,Irvine,CA,3.748048,,
4,Alexandria,VA,3.738484,,
...,...,...,...,...,...
177,Surprise,AZ,-2.594253,,
178,Tempe,AZ,-2.927363,,
179,Fayetteville,NC,-3.271293,,
180,Midland,TX,-3.421076,,


In [31]:
def getlatlng(params): #creating a function to get coordinates 
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    response = requests.get(base_url, params=params)
    data = response.json()
    lat = data['results'][0]['geometry']['location']['lat']
    lng = data['results'][0]['geometry']['location']['lng']
    return lat, lng

In [33]:
for index, row in pull_pop.iterrows():
    print(f'working on {index} row')
    params = {
            "key": gkey,
            'address' : row['City'] + ',' + row['State']
             }
    lat, lng = getlatlng(params)
    try:
        pull_pop.loc[index,"Lat"] = lat
        pull_pop.loc[index,"Lng"] = lng
    except:
        print(f"missing value for row number{index}")

working on 0 row
working on 1 row
working on 2 row
working on 3 row
working on 4 row
working on 5 row
working on 6 row
working on 7 row
working on 8 row
working on 9 row
working on 10 row
working on 11 row
working on 12 row
working on 13 row
working on 14 row
working on 15 row
working on 16 row
working on 17 row
working on 18 row
working on 19 row
working on 20 row
working on 21 row
working on 22 row
working on 23 row
working on 24 row
working on 25 row
working on 26 row
working on 27 row
working on 28 row
working on 29 row
working on 30 row
working on 31 row
working on 32 row
working on 33 row
working on 34 row
working on 35 row
working on 36 row
working on 37 row
working on 38 row
working on 39 row
working on 40 row
working on 41 row
working on 42 row
working on 43 row
working on 44 row
working on 45 row
working on 46 row
working on 47 row
working on 48 row
working on 49 row
working on 50 row
working on 51 row
working on 52 row
working on 53 row
working on 54 row
working on 55 row
wo

In [34]:
pull_pop

,City,State,Diff in Pop Rate 2018-2020,Lat,Lng
0,Salt Lake City,UT,6.316051,40.7608,-111.891
1,Hialeah,FL,4.911342,25.8576,-80.2781
2,Jersey City,NJ,4.116396,40.7178,-74.0431
3,Irvine,CA,3.748048,33.6846,-117.827
4,Alexandria,VA,3.738484,38.8048,-77.0469
...,...,...,...,...,...
177,Surprise,AZ,-2.594253,33.6292,-112.368
178,Tempe,AZ,-2.927363,33.4255,-111.94
179,Fayetteville,NC,-3.271293,35.0527,-78.8784
180,Midland,TX,-3.421076,31.9973,-102.078


In [45]:
positive_pop=pull_pop.iloc[:123,:]

In [82]:
locations = positive_pop[['Lat','Lng']]
weight = positive_pop['Diff in Pop Rate 2018-2020']*100


In [83]:
# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig= gmaps.figure(layout=figure_layout, zoom_level = 3.8, center = (37.09,-95.71))

# Add Heatmap layer to map that displays pop rate% change for every city in dataset
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=150,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…